In [151]:
import pandas as pd
import numpy as np

In [152]:
data_train = pd.read_csv('data/train.csv')
data_test = pd.read_csv('data/kaggle_test_data.csv')

In [153]:
features = list(data_train)[1:-1]
target = list(data_train)[-1]

In [154]:
strings = []
for feature in features:
    if data_train[feature].dtype == 'object':
        curr_mode = data_train[feature].mode()[0]
        data_train[feature] = data_train[feature].apply(lambda x :
                                                       curr_mode if x == " ?" else x)

In [155]:
# one hot encoding
for feature in features:
    if data_train[feature].dtype == "object":
        values = data_train[feature].unique()
        for value in values:
            data_train[value] = data_train[feature].apply(lambda z : 1 if z == value else 0)
        

In [156]:
features = list(set(list(data_train)) - set(features+[target,"id"]))
#print features

In [157]:
input_matrix = data_train[features].as_matrix()
target_matrix = data_train[target].as_matrix()
print input_matrix.shape
print target_matrix.shape

(38973, 99)
(38973,)


In [158]:
def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

In [159]:
def sigmoid_prime(output):
    return output*(1.0-output)

In [232]:
def neural_net(n_inputs,n_hidden,n_output,data,target):
    np.random.seed(1)
    
    theta0 = 2*np.random.random((n_inputs+1,n_hidden)) - 1
    theta1 = 2*np.random.random((n_hidden+1,n_output)) - 1
    
    print "Theta0 is : ",theta0.shape
    print "Theta1 is : ",theta1.shape
    
    layer0 = data
    layer0 = np.c_[np.ones(len(layer0)),layer0]
    print "Layer 0 is : ",layer0.shape
    layer1 = sigmoid(np.dot(data,theta0))
    layer1 = np.c_[np.ones(len(layer1)),layer1]
    #print layer1.shape
    print "Layer 1 is : ",layer1.shape
    layer2 = sigmoid(np.dot(layer1,theta1))
    print "Layer 2 is : ",layer2.shape
    
    layer2_error = (target - layer2.T).T
    layer2_delta = layer2_error*sigmoid_prime(layer2)
    print "Layer 2 error is : ",layer2_error.shape
    print "Layer 2 delta is : ",layer2_delta.shape
    
    layer1_error = np.dot(layer2_delta,theta1.T)
    layer1_error = layer1_error[:,1:]
    print "Layer 1 error is : ",layer1_error.shape
    
    layer1_delta = layer1_error*sigmoid_prime(layer1)
    print "Layer 1 delta is : ",layer1_delta.shape
    
    print layer0.T.dot(layer1_delta).shape
    #theta1 += layer1.T.dot(layer2_delta)
    print layer1.T.dot(layer2_delta).shape
    
    
    
    
    

In [233]:
neural_net(99,100,1,input_matrix,target_matrix)

Theta0 is :  (100, 100)
Theta1 is :  (101, 1)
Layer 0 is :  (38973, 100)


ValueError: shapes (38973,99) and (100,100) not aligned: 99 (dim 1) != 100 (dim 0)

In [31]:
x = np.array([[0,0],[0,0],[2,2]])

In [33]:
x = x.

3